## TRAINING
This notebook has as an objective to use a folder containing the formated .npy files to train a deep learning model that can be used in lieu of an FEA model to any accuracy above random, for a proof of concept that it is possible to do so, such that further research can be done afterward to optimize architecture, hyperparameters and data being fed in.

In [68]:
##  imports

import numpy as np
import torch
import torch.nn as nn
import pathlib
import PREPROCESSING_splitting as split

In [87]:
## first, create a class to load the files that are to be fed to the neural network, 
## for both the inputs and the outputs. To avoid confusion I'll refer to the inputs to the 
## FEA model as 'bound_conds' (boundary conditions), and the outputs as 'targets', while what 
## is fed into the neural network will be called an "input", and the output of the neural network "prediction"
## this function should probably be transformed into a dataloader later for a larger dataset, 
## but for now we'll keep it like this

def get_dataset(dataset_path, glob_parameter = '*.npy'):
    # concatenates all samples into a list of boundary conditions and a list of targets
    
    # set paths
    bound_cond_path = pathlib.Path(dataset_path, 'input')
    targets_path = pathlib.Path(dataset_path, 'output')

    test = pathlib.Path('D:/')
    
    
    
    
    # check if folder path is correct
    if bound_cond_path.is_dir() and targets_path.is_dir():
        print('path contains \'input\' and \'output\'')
        pass
    else:
        raise Exception (f'Argument dataset_path: {dataset_path} should contain folders ..\input and ..\output. Please check path')
    
    #create iterators for files
    bound_cond_iterator = bound_cond_path.glob(glob_parameter)
    targets_iterator = targets_path.glob(glob_parameter)
    
    #zip them to ensure that they are going through the same samples 
    samples_iterator = zip(bound_cond_iterator, targets_iterator)
    
    boundary_conditions = np.array([])
    targets = np.array([])
    
    for boundary_condition_files, targets_files in samples_iterator:
        if split.get_number(boundary_condition_files.name) == split.get_number(targets_files.name):
            
            boundary_conditions_temp = np.load(boundary_condition_files)
            targets_temp = np.load(targets_files)
            
            #start array if it hasn't been started yet
            if boundary_conditions.size == 0 and targets.size == 0:
                boundary_conditions = boundary_conditions_temp
                targets = targets_temp
            else:
                boundary_conditions = np.concatenate((boundary_conditions, boundary_conditions_temp), axis = 0)
                targets = np.concatenate((targets, targets_temp), axis = 0)
        else:
            raise Exception('the samples in the iterator are not synced')
    
    return torch.from_numpy(boundary_conditions).float(), torch.from_numpy(targets).float()
    

In [88]:
folders_path =  pathlib.Path('D:/Ansys Simulations/Project/2D/data/proof_of_concept/scaled/arrays')
dataset = get_dataset(dataset_path = folders_path)
print(dataset[0].shape)
print(dataset[1].shape)

path contains 'input' and 'output'
torch.Size([102, 7, 32, 32])
torch.Size([102, 4, 32, 32])


In [91]:
## define the neural network's general shape

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        ## convolutional layers
        self.conv1 = nn.Conv2d(in_channels = 7, out_channels = 14, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 14, out_channels = 16, kernel_size = 3, padding = 1)
        self.deconv1 = nn.ConvTranspose2d(in_channels = 16, out_channels = 14, kernel_size = 3, padding = 1)
        self.deconv2 = nn.ConvTranspose2d(in_channels = 14, out_channels = 10, kernel_size = 3, padding = 1)
        self.deconv3 = nn.ConvTranspose2d(in_channels = 10, out_channels = 7, kernel_size = 3, padding = 1)
        self.deconv4 = nn.ConvTranspose2d(in_channels = 7, out_channels = 3, kernel_size = 3, padding = 1)
        
        ## activation
        self.hardtanh = nn.Hardtanh()
        
        
        ##possible for later: MultiheadAttention
        
    def forward(self, boundary_conditions):
        print(boundary_conditions.shape)
        x = self.conv1(boundary_conditions)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.conv2(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv1(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv2(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv3(x)
        x = self.hardtanh(x)
        print(x.shape)
        x = self.deconv4(x)
        x = self.hardtanh(x)
        print(x.shape)
        return x
    
net = ConvNet().float()

## test to see if getting the correct size
net.forward(dataset[0][0:1,:,:,:])

torch.Size([1, 7, 32, 32])
torch.Size([1, 14, 32, 32])
torch.Size([1, 16, 32, 32])
torch.Size([1, 14, 32, 32])
torch.Size([1, 10, 32, 32])
torch.Size([1, 7, 32, 32])
torch.Size([1, 3, 32, 32])


tensor([[[[ 0.0591,  0.0511,  0.0485,  ...,  0.0385,  0.0486,  0.0397],
          [ 0.0462,  0.0245,  0.0104,  ..., -0.0054,  0.0102,  0.0179],
          [ 0.0335,  0.0164,  0.0012,  ..., -0.0124,  0.0033,  0.0166],
          ...,
          [ 0.0797,  0.0754,  0.0406,  ..., -0.0079,  0.0116,  0.0210],
          [ 0.0763,  0.0732,  0.0416,  ...,  0.0043,  0.0214,  0.0216],
          [ 0.0419,  0.0440,  0.0163,  ...,  0.0052,  0.0110,  0.0320]],

         [[-0.0301, -0.0742, -0.0626,  ..., -0.0729, -0.0654, -0.0790],
          [-0.0292, -0.0947, -0.0963,  ..., -0.1063, -0.1130, -0.1121],
          [-0.0147, -0.0895, -0.0957,  ..., -0.1042, -0.1094, -0.1091],
          ...,
          [ 0.0040, -0.0509, -0.0415,  ..., -0.0947, -0.0993, -0.1022],
          [ 0.0064, -0.0695, -0.0642,  ..., -0.1030, -0.1089, -0.1090],
          [-0.0155, -0.0533, -0.0523,  ..., -0.0852, -0.0889, -0.0764]],

         [[ 0.1804,  0.1742,  0.1745,  ...,  0.1702,  0.1765,  0.1904],
          [ 0.2035,  0.1667,  

In [ ]:
## define a general training loop

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')